In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns
from sklearn.neighbors import KernelDensity
from scipy.optimize import minimize

In [2]:
# Definition of the Marchenko-Pastur density
def marchenko_pastur_pdf(x,Q,sigma=1):
    y=1/Q
    b=np.power(sigma*(1 + np.sqrt(1/Q)),2) # Largest eigenvalue
    a=np.power(sigma*(1 - np.sqrt(1/Q)),2) # Smallest eigenvalue
    if x > b or x<a:
        pdf = 0
    else:
        pdf = (1/(2*np.pi*sigma*sigma*x*y))*np.sqrt((b-x)*(x-a))
        
    return pdf

    #return (1/(2*np.pi*sigma*sigma*x*y))*np.sqrt((b-x)*(x-a))*(0 if (x > b or x <a ) else 1)
    
def compare_eigenvalue_distribution(correlation_matrix, Q, sigma=1, set_autoscale = True, show_top = True,show=True):
    e, _ = np.linalg.eig(correlation_matrix) # Correlation matrix is Hermitian, so this is faster
                                   # than other variants of eig
    
    x_min = .0001 if np.power(sigma*(1 - np.sqrt(1/Q)),2) < .0001 else np.power(sigma*(1 - np.sqrt(1/Q)),2)
    x_max = np.power(sigma*(1 + np.sqrt(1/Q)),2)
            
    fig = plt.figure()
    ax  = fig.add_subplot(111)
    bins = 50
    if not show_top:
            # Clear top eigenvalue from plot
        e=e[ e <= x_max+1]
    sns.histplot(e, stat='density', bins=50, ax=ax)
    #ax.hist(e, density = True, bins=100) # Histogram the eigenvalues
    ax.set_autoscale_on(set_autoscale)
        
        # Plot the theoretical density
    f = np.vectorize(lambda x : marchenko_pastur_pdf(x,Q,sigma=sigma))
        
    x_min = .0001 if np.power(sigma*(1 - np.sqrt(1/Q)),2) < .0001 else np.power(sigma*(1 - np.sqrt(1/Q)),2)
    x_max = np.power(sigma*(1 + np.sqrt(1/Q)),2)
        
    x = np.linspace(x_min,x_max,5000)
    ax.plot(x,f(x), linewidth=4, color = 'r')
    ax.set_xlim((0,10))

In [3]:
def fitKDE(obs, bWidth=0.25, kernel='gaussian', x=None):
    if len(obs.shape)==1:
        obs =  obs.reshape(-1,1)
    kde =  KernelDensity(kernel=kernel, bandwidth=bWidth).fit(obs)
    if x is None:
        x = np.unique(obs).reshape(-1,1)
    if len(x.shape)==1:
        x = x.reshape(-1,1)
    logProb = kde.score_samples(x) # Log(density)
    pdf = pd.Series(np.exp(logProb).flatten(), index=x.flatten())
    return pdf

In [4]:
def compare_eigenvalue_distribution_notshow(correlation_matrix, Q, sigma=1, set_autoscale = True, show_top = True,show=True):
    e, _ = np.linalg.eig(correlation_matrix) # Correlation matrix is Hermitian, so this is faster
                                   # than other variants of eig
    
    x_min = .0001 if np.power(sigma*(1 - np.sqrt(1/Q)),2) < .0001 else np.power(sigma*(1 - np.sqrt(1/Q)),2)
    x_max = np.power(sigma*(1 + np.sqrt(1/Q)),2)

    if not show_top:
            # Clear top eigenvalue from plot
        e=e[ e <= x_max+1]
        # Plot the theoretical density
    f = np.vectorize(lambda x : marchenko_pastur_pdf(x,Q,sigma=sigma))
        
    x_min = .0001 if np.power(sigma*(1 - np.sqrt(1/Q)),2) < .0001 else np.power(sigma*(1 - np.sqrt(1/Q)),2)
    x_max = np.power(sigma*(1 + np.sqrt(1/Q)),2)
    return e, x_max, x_min


In [5]:
def pdf_mp(var,q,pts,cor):
    eVal,eMax, eMin  = compare_eigenvalue_distribution_notshow(cor, q)
    x = np.linspace(eMin,eMax, pts)
    f = np.vectorize(lambda x : marchenko_pastur_pdf(x,q,var))
    pdf = pd.Series(f(x).flatten(),
                   index=x.flatten())
    return pdf

In [6]:
def errPDFs(x,eVal,bandwidth,cor, pts=100):
    
    pdf0 = pdf_mp(x[0],x[1],pts,cor=cor)
    
    pdf1 = fitKDE(obs=eVal,
                      bWidth=bandwidth,
                      x=pdf0.index.values)

    sse = np.sum((pdf1-pdf0)**2)
    return sse

In [7]:
def findMaxEval(eVal,bWidth,cor):
    # Find max random eVal by fitting Marcenko’s dist
    out=minimize(fun= errPDFs,
                 x0=(1,12),
                 args=(eVal,bWidth,cor),
                 bounds=((0,None),
                        (0,None)))
    result = out
    
    if out['success']:
        var=out['x']#[0]
        print('succes')
    else:
        var=1
    #eMax=var[0]*(1+(1./var[1])**.5)**2
    return var, result